#### Cassandra Python Programming Assignment 
##### Gayathri Shanmuga Sundaram - U88629479
INSTRUCTIONS:
1. Connect to the Cassandra Database
2. Create a keyspace called m14
3. Create a table in this keyspace that will be used to store inventory information. This will include the fields, SKU (which is shown for stock keeping unit), name (short name for the product), description (longer description of the product), warehouse_num (the warehouse number where the product is stored)
4. Populate several rows of data into your inventory table
5. Create an index on warehouse_num. 
6. Search for all products located at a given location (for instance - if you created two rows with location 123, list all products at location 123)

Importing Packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

Loading Data from csv file

In [2]:
df = pd.read_csv('./toystore_inventory.csv')
df

,sku,name,description,warehouse_num
0,'SK4567A','AMEROUS','15 Inches Magnetic Wooden Chess Set - Beginn...,1433
1,'PH6788G','Taco vs Burrito','The Strategic Family Friendly Card Game Crea...,6677
2,'YU4456Z','Wingspan Board Game','A Bird-Collection Engine-Building STONEMAIER...,8760
3,'NZ6789T','Skillmatics Card Game','Found It Indoor Edition Scavenger Hunt for K...,6677
4,'GY3452I','Azul-Board Game Strategy','Board Game Mosaic-Tile Placement Game Family...,1433


Create a session connection to Cassandra cluster

In [3]:
clstr = Cluster()
session = clstr.connect()

Create keyspace

In [4]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}")

Veryfing keyspace creation

In [5]:
rows = session.execute("desc keyspaces")

In [6]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04
w04python


Create table

In [7]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.toystore ( 
    sku TEXT, 
    name TEXT, 
    description TEXT,  
    warehouse_num INT, 
    PRIMARY KEY(sku)
);
""")

Inserting the Row values

In [8]:
for index, row in df.iterrows():
    session.execute(f"""
        INSERT INTO m14.toystore (sku, name, description, warehouse_num)     
        VALUES ({row[0].strip()}, {row[1].strip()}, {row[2].strip()}, {row[3]});
        """
       )

Create INDEX

In [9]:
session.execute ("CREATE INDEX ON m14.toystore(warehouse_num)")

Run the Query

In [10]:
rows = session.execute("select (sku, name, description, warehouse_num) from m14.toystore where warehouse_num = 6677")
for row in rows:
    print(f"sku={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]},")

sku=PH6788G, name=Taco vs Burrito, description=The Strategic Family Friendly Card Game Created by a 7 Year Old, warehouse_num=6677,
sku=NZ6789T, name=Skillmatics Card Game, description=Found It Indoor Edition Scavenger Hunt for Kids Fun Family Game Ages 4 to 7, warehouse_num=6677,
